In [ ]:
import geopandas as gpd
import networkx as nx
import osmnx as ox
import shapely
import network_model
import folium

Инициализация сетевой модели. Загрузка геослоя с площадями и слоя с домами в класс

In [ ]:
OPM_path = "/Users/mvin/Desktop/RNF_DEV/code_v2/DATA/potential_ps.geojson"
house_path = "/Users/mvin/Desktop/RNF_DEV/code_v2/DATA/House.geojson"

model = network_model.NetworkModel(OPM_path=OPM_path, house_path=house_path)
model.OPM

Поиск и классификация зеленых зон

In [ ]:
model.find_exist("/Users/mvin/Desktop/RNF_DEV/code_v2/DATA/service.geojson")
model.OPM

In [ ]:
model.OPM[model.OPM["functional_type"] != ""]

Вызов метода для построения изохрон. На вход подается граф пешей доступности

In [ ]:
model.create_isochrones("/Users/mvin/Desktop/RNF_DEV/code_v2/DATA/walk_graph_spb_32636.graphml")
model.iso_5

Создание графа

In [ ]:
g = model.create_graph()
g

In [ ]:
model.OPM_info()
model.provision_info()

выбор одного оптимального места для ОПМ заданного размера и функционального типа, без открытия ОПМ в этом месте

In [ ]:
a = model.best_location1("Событийное", "medium")
print(f"id пространства: {a[1]}")
g.nodes[a[1]]

Вызов функций для нахождения полного решения. Сначала вызывается метод full_solution(), далее - local_search() для улучшения получившегося решения

In [ ]:
model.full_solution()

In [ ]:
model.local_search()

In [ ]:
model.OPM_info()
model.provision_info()

Рекомендательная функция

In [ ]:
model.recommend_services("Многофункциональное", 200)

In [ ]:
opm = model.geo()
house = model.get_houses()

In [ ]:
h = model.get_houses()

In [ ]:
# h.to_file('house_0.geojson', driver = 'GeoJSON')

In [ ]:
# opm.to_file('opm.geojson', driver='GeoJSON')

In [ ]:
m = folium.Map(location=[59.938732, 30.316229], zoom_start=10, tiles="CartoDB positron")

for i, r in opm.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    if r["functional_type"] == "Многофункциональное":
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "blue"})
    if r["functional_type"] == "Событийное":
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "red", "color": "red"})
    if r["functional_type"] == "Спортивное":
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "green", "color": "green"})
    if r["functional_type"] == "Учебное":
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "yellow", "color": "yellow"})
    folium.Popup(r["ratio"]).add_to(geo_j)
    geo_j.add_to(m)
m